# Cryoscope

Alternate experiment to characterise the pulse distortions from line impedance, following https://arxiv.org/pdf/1907.04818.pdf - Ramsey sequence with fixed timing and variable flux pulse in between - sweeping flux pulse length and amplitude

## 0. General Imports and Definitions

### 0.1 Python Imports

In [ ]:
# LabOne Q:
from laboneq.contrib.example_helpers.example_notebook_helper import create_device_setup

# Helpers:
from laboneq.contrib.example_helpers.plotting.plot_helpers import *
from laboneq.simple import *

## 1. Device Setup

In [ ]:
# create device setup
device_setup = create_device_setup(generation=2)
use_emulation = True

In [ ]:
#  signal map for qubits
def map_qubit(qubit_id):
    return {
        "drive": f"/logical_signal_groups/q{qubit_id}/drive_line",
        "flux": f"/logical_signal_groups/q{qubit_id}/flux_line",
        "measure": f"/logical_signal_groups/q{qubit_id}/measure_line",
        "acquire": f"/logical_signal_groups/q{qubit_id}/acquire_line",
    }

## 2. Experiment Definition

In [ ]:
## define standard pulses

# qubit drive pulse - pi/2 rotation
x90 = pulse_library.gaussian(uid="x90", length=100e-9, amplitude=0.66)
# readout drive pulse
readout_pulse = pulse_library.const(uid="readout_pulse", length=250e-9, amplitude=1.0)
# readout weights for integration
readout_weighting_function = pulse_library.const(
    uid="readout_weighting_function", length=200e-9, amplitude=1.0
)

In [ ]:
# define near time sweep parameter - pulse length
length_start = 10e-9
length_stop = 100e-9
length_count = 5  # known issue: limited to max. 30 steps

flux_length_sweep = LinearSweepParameter(
    start=length_start, stop=length_stop, count=length_count
)

flux_pulse = pulse_library.const(uid="flux_pulse", length=length_start, amplitude=1.0)

In [ ]:
# define flux amplitude sweep parameter
flux_amp_count = 7

flux_amp_sweep = LinearSweepParameter(
    uid="flux_qubit", start=0.1, stop=1, count=flux_amp_count
)

# define number of averages
average_exponent = 1  # used for 2^n averages, n=average_exponent, maximum: n = 17

### 2.1 Pulse Sequence

In [ ]:
# Create Experiment - no explicit mapping to qubit lines
exp = Experiment(
    uid="Cryoscope experiment",
    signals=[
        ExperimentSignal("flux"),
        ExperimentSignal("drive"),
        ExperimentSignal("measure"),
        ExperimentSignal("acquire"),
    ],
)

with exp.acquire_loop_rt(
    uid="shots",
    count=pow(2, average_exponent),
    averaging_mode=AveragingMode.CYCLIC,
    acquisition_type=AcquisitionType.INTEGRATION,
):
    with exp.sweep(parameter=flux_amp_sweep):
        with exp.sweep(parameter=flux_length_sweep):
            # qubit excitation pulses - Ramsey with fixed timing
            with exp.section(uid="ramsey"):
                # play first Ramsey excitation pulse
                exp.play(signal="drive", pulse=x90)
                exp.delay(signal="drive", time=2 * length_stop)
                # play second Ramsey excitation pulse
                exp.play(signal="drive", pulse=x90)
            # interleaved flux pulse with variable length and amplitude
            with exp.section(uid="flux"):
                # delay while first Ramsey pulse is played
                exp.delay(signal="flux", time=x90.length)
                # flux pulse
                exp.play(
                    signal="flux",
                    pulse=flux_pulse,
                    amplitude=flux_amp_sweep,
                    length=flux_length_sweep,
                )
            # readout and data acquisition
            with exp.section(uid="qubit_readout"):
                exp.reserve(signal="drive")
                exp.reserve(signal="flux")
                # play readout pulse
                exp.play(signal="measure", pulse=readout_pulse)
                # signal data acquisition
                exp.acquire(
                    signal="acquire",
                    handle="ac_0",
                    kernel=readout_weighting_function,
                )
            # relax time after readout - for signal processing and qubit relaxation to ground state
            with exp.section(uid="relax", length=1e-6):
                exp.reserve(signal="measure")

### 2.2 Run the Experiment and Plot the Measurement Results and Pulse Sheet

In [ ]:
# set calibration and signal map for qubit 0
exp.set_signal_map(map_qubit(0))

# create a session and connect to it
session = Session(device_setup=device_setup)
session.connect(do_emulation=use_emulation)

my_results = session.run(exp)

In [ ]:
# Plot simulated output signals
plot_simulation(session.compiled_experiment, start_time=0, length=10e-6)

In [ ]:
# plot measurement results
plot_result_3d(my_results, "ac_0")

In [ ]:
# use pulse sheet viewer to display the pulse sequence - only recommended for small number of averages and sweep steps to avoid performance issues
show_pulse_sheet("Cryoscope", session.compiled_experiment)

In [ ]:
# set calibration and signal map for qubit 1
exp.set_signal_map(map_qubit(1))

# create a session and connect to it
session = Session(device_setup=device_setup)
session.connect(do_emulation=use_emulation)

my_results = session.run(exp)